In [43]:
#Расчёт пластинчатого ТО (волны-sandia)
import matplotlib.pyplot as pt
import numpy as n
from CoolProp.CoolProp import PropsSI as p
from scipy.optimize import root
import math as m

#Исходные данные:
G1 = 127 #горяч
T11 = 64
T12 = 41

G2 = 127
T21 = 36



P11=6400000
P21=10500000
P12 = P11
P22 = P21

##############################################
CO2  = 'REFPROP::CO2'
H2O  = 'REFPROP::WATER'
O2   = 'REFPROP::Oxygen'
MIX1 =  'REFPROP::CO2[0.931]&WATER[0.069]'
MIX2 =  'REFPROP::CO2[0.993]&WATER[0.007]'
MIX05 =  'REFPROP::CO2[0.95]&WATER[0.05]' #Для транспортный свойств


H11  = p('H','T', T11+273.15, 'P', P11, CO2) #заменить на СО2
H12  = p('H','T', T12+273.15, 'P', P11, CO2)
H21  = p('H','T', T21+273.15, 'P', P21, CO2)

#Разбиение по энтальпии:
s    = 100 #Кол-во шагов
step = (H11-H12)/(s) 

#Нулевые массивы:
t1   = n.zeros(s+1)
t2   = n.zeros(s+1)
Q    = n.zeros(s+1)

#Противоток:
h11 = H11
i=0
for i in range(s+1):
    t1[i] = p('T','H', h11, 'P', P11, CO2)-273.15 
    h12 = h11 - step
    dQ = G1 * (h11-h12)
    if i < s:
        Q[i+1]=Q[i]+dQ
    h11 = h12
h21 = H21
i=0
for i in range(s+1):
    t2[s-i] = p('T','H', h21, 'P', P21, CO2)-273.15  
    h22 = h21+((Q[s-i]-Q[s-i-1])/G2)
    h21 = h22
DT=t1-t2
##############################################
print(min(DT))

t1av = n.zeros(s)
t2av = n.zeros(s)
ro1 = n.zeros(s)
ro2 = n.zeros(s)
nu1 = n.zeros(s)
nu2 = n.zeros(s)
lambda1 = n.zeros(s)
lambda2 = n.zeros(s)
Pr1 = n.zeros(s)
Pr2 = n.zeros(s)
DTav = n.zeros(s)
w1 = n.zeros(s)
w2= n.zeros(s)
Re1= n.zeros(s)
Re2= n.zeros(s)
Nu1= n.zeros(s)
Nu2= n.zeros(s)
f1= n.zeros(s)
f2= n.zeros(s)
alpha1= n.zeros(s)
alpha2= n.zeros(s)
K= n.zeros(s)
F= n.zeros(s+1)
L= n.zeros(s+1)
P1= n.zeros(s+1)
P2= n.zeros(s+1)


ddP1     =n.zeros(10000)
ddP2     =n.zeros(10000)
ddP4     =n.zeros(10000)
ddP6     =n.zeros(10000)
ddP8     =n.zeros(10000)
ddP10    =n.zeros(10000)




Vv     =n.zeros(10000000)
DdP1   =n.zeros(10000000)
DdP2   =n.zeros(10000000)
Ff     =n.zeros(10000000)
Ll     =n.zeros(10000000)
Nnsumm =n.zeros(10000000)

#Для каждого участка:
for i in range(s):
    #Свойства
    t1av[i]   =(t1[i]+t1[i+1])/2
    t2av[i]   =(t2[i]+t2[i+1])/2
    ro1[i]    =p('D','T', t1av[i]+273.15, 'P', P11, CO2)
    ro2[i]    =p('D','T', t2av[i]+273.15, 'P', P21, CO2)
    nu1[i]    =p('V','T', t1av[i]+273.15, 'P', P11, CO2)/ro1[i]
    nu2[i]    =p('V','T', t2av[i]+273.15, 'P', P21, CO2)/ro2[i]
    lambda1[i]=p('L','T', t1av[i]+273.15, 'P', P11, CO2)
    lambda2[i]=p('L','T', t2av[i]+273.15, 'P', P21, CO2)
    Pr1[i]    =p('PRANDTL','T', t1av[i]+273.15, 'P', P11, CO2)
    Pr2[i]    =p('PRANDTL','T', t2av[i]+273.15, 'P', P21, CO2)
    DTav[i]   =t1av[i]-t2av[i]
#От выпадающих Pr:
for i in range(1,s):
    if abs(Pr1[i]-Pr1[i-1])>Pr1[i-1]:
        Pr1[i]=p('PRANDTL','T', t1[i]+273.15, 'P', P11, CO2)
    if abs(Pr2[i]-Pr2[i-1])>Pr2[i-1]:
        Pr2[i]=p('PRANDTL','T', t2[i]+273.15, 'P', P21, CO2)

#Геометрия
delta = 0.002       #толщина металла
wpl = 1             #ширина пластины
fprohc = 8.8718*(10**-5)*74
hc=0.01
dh = 0.00624         #характерный размер
hpl = hc+delta      #высота пластины
lambdaw = 16        #теплопроводность металла
fl = 3.7

#Геометрия "выхода":
roout1 = p('D','T', t1[s]+273.15, 'P', P11, CO2)
roout2 = p('D','T', t2[s]+273.15, 'P', P21, CO2)


print('+')
j=0


wout1= 2.2 #заменить


Fproh1 = G1/(roout1*wout1)
Npl1 = m.ceil(Fproh1/fprohc)
Npl2 = m.ceil(Npl1)
Nsumm = Npl1+Npl2
Fproh2 = Npl2*fprohc
Fl = ((Npl1+Npl2)/2)*fl


P1[0]=P11
P2[0]=P21
#Для каждого участка:
for i in range(s):
    w1[i] = G1/(ro1[i]*Fproh1)
    w2[i] = G2/(ro2[i]*Fproh2)
    Re1[i]=w1[i]*dh/nu1[i]
    Re2[i]=w2[i]*dh/nu2[i]
    Nu1[i] = 0.0057*Re1[i]**(0.9926)  #заменить
    Nu2[i] = 0.0057*Re2[i]**(0.9926)
    f1[i] = 59107*Re1[i]**(-1.292)
    f2[i] = 59107*Re2[i]**(-1.292)
    alpha1[i] = Nu1[i]*lambda1[i]/dh
    alpha2[i] = Nu2[i]*lambda2[i]/dh
    
    K[i] = 1/((1/alpha1[i])+(delta/lambdaw)+(1/alpha2[i]))
    dF = (Q[i+1]-Q[i])/(K[i]*DTav[i])
    dL = dF/Fl
    if i < s:
        F[i+1]=F[i]+dF
    if i < s:
        L[i+1]=L[i]+dL
    dP1 = f1[i]*dL/dh*ro1[i]*w1[i]**2/2
    dP2 = f2[i]*dL/dh*ro2[i]*w2[i]**2/2
    if i < s:
        P1[i+1]=P1[i]-dP1
        P2[i+1]=P2[i]-dP2
    DP1 = P1[0]-P1[s]
    DP2 = P2[0]-P2[s]
print("++")
print('Площадь','Кол-во пластин','Длина')
print(F[s], Nsumm,L[s], DP1, DP2)
#V = #вставишь
#row = 7880 #плотность
#cost1kg = 0.97*75 #
#COST = V*row*cost1kg*1.4

#256k

4.999999995350265
+
++
Площадь Кол-во пластин Длина
423.075981701293 108 2.117497405912377 2309.309180271812 2006.205169826746
